# **Implementación de un Transformer Decoder tipo GPT para Generación de Texto**

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
import numpy as np

print("TensorFlow version:", tf.__version__)

## **Tokenización letra por letra (caracter a entero)**

La tokenización del texto se realiza a nivel de **caracteres**, no de palabras ni n-conjuntos de palabras. El objetivo es asignar a cada carácter único un identificador numérico entero, usando un mapeo basado en el orden alfabético.

---

### ¿Cómo funciona?

1. **Obtener los caracteres únicos del corpus**
   - Se extraen todos los caracteres únicos presentes en el texto.
   - Por ejemplo, si el texto es `"Hola."`, los caracteres únicos serían `['H', 'o', 'l', 'a','.']`.

2. **Ordenar alfabéticamente**
   - Los caracteres únicos se ordenan de forma alfabética para asignar los índices de forma determinista.
   - Ejemplo: `['.','H', 'a', 'l', 'o']`

3. **Mapeo carácter a índice**
   - Se crea un diccionario `char2idx` que asigna a cada carácter un número entero.
   - Por ejemplo:
     ```python
     char2idx = {'.': 0, 'H': 1, 'a': 2, 'l': 3, 'o': 4}
     ```

4. **Mapeo inverso índice a carácter**
   - También se crea `idx2char` para decodificar las predicciones del modelo:
     ```python
     idx2char = {0: '.', 1: 'H', 2: 'a', 3: 'l', 4: 'o'}
     ```

---


In [ ]:
# Texto corto para prueba
text = (
    "La National Football League (NFL) es la principal liga profesional de fútbol americano en los Estados Unidos. "
    "Fue fundada en 1920 como la American Professional Football Association y cambió su nombre a NFL en 1922. "
    "Está compuesta por 32 equipos divididos en dos conferencias: la American Football Conference (AFC) y la National Football Conference (NFC). "
    "Cada conferencia tiene cuatro divisiones: Norte, Sur, Este y Oeste. "
    "Los equipos compiten durante una temporada regular de 17 semanas para clasificar a los playoffs, que culminan en el Super Bowl, "
    "uno de los eventos deportivos más vistos a nivel mundial. "

    "Equipos legendarios como los Pittsburgh Steelers, con seis títulos de Super Bowl, y los New England Patriots, también con seis campeonatos, "
    "han dejado una huella imborrable en la historia del deporte. "
    "Franquicias como los Dallas Cowboys son conocidos como 'America’s Team', mientras que los Green Bay Packers, con su base de fans comunitaria, "
    "son famosos por su legado y tradición. "

    "El fútbol americano se juega entre dos equipos de 11 jugadores cada uno. "
    "El objetivo es avanzar el balón en la zona de anotación del equipo rival, lo cual se puede lograr por tierra (acarreos) o por aire (pases). "
    "Cada equipo tiene cuatro oportunidades (downs) para avanzar 10 yardas. "
    "Si lo logran, obtienen una nueva serie de downs. Si no, deben despejar (punt) o intentar un gol de campo si están en posición. "

    "El mariscal de campo o quarterback (QB) es la pieza clave de la ofensiva. Figuras como Tom Brady, Peyton Manning y Patrick Mahomes "
    "han redefinido el papel del QB en distintas épocas. "
    "En la defensa, jugadores como Ray Lewis, J.J. Watt y Aaron Donald han dominado con su capacidad para leer jugadas y detener al rival. "

    "Las posiciones se dividen en ofensiva (QB, RB, WR, TE, OL), defensiva (DL, LB, CB, S) y equipos especiales (K, P, LS, KR, PR). "
    "El reglamento incluye penalizaciones como interferencia de pase, sujeción y formación ilegal. "
    "El reloj de juego, el manejo de tiempos fuera y las estrategias de reloj son fundamentales en los últimos minutos del partido. "

    "La NFL celebra eventos clave como el Draft, el Combine, el Pro Bowl y el Super Bowl. "
    "Además, cuenta con iniciativas de responsabilidad social, como 'Crucial Catch' para la lucha contra el cáncer, y 'My Cause My Cleats', "
    "que permite a los jugadores expresar causas personales. "

    "La rivalidad entre equipos es una parte esencial de la cultura de la NFL. Enfrentamientos como Packers vs. Bears, Cowboys vs. Eagles, "
    "y Patriots vs. Jets tienen décadas de historia e intensidad. "

    "El Super Bowl LVIII se llevó a cabo en el Allegiant Stadium de Las Vegas, Nevada. Patrick Mahomes lideró a los Kansas City Chiefs a una victoria ajustada, "
    "confirmando su estatus como uno de los mejores quarterbacks de la nueva generación. "

    "Con transmisiones en más de 180 países y una base de fans global en expansión, la NFL continúa siendo un símbolo de estrategia, potencia, "
    "resistencia y emoción cada domingo. "

    "Desde los estadios llenos de historia como Lambeau Field y Arrowhead Stadium, hasta las modernas instalaciones como SoFi Stadium, "
    "el espectáculo del fútbol americano combina tradición con innovación. "
    "Miles de analistas, comentaristas y fanáticos discuten semana a semana cada jugada, cada decisión de los entrenadores y cada trade en la liga. "
    "La fantasy football ha creado una nueva forma de interacción entre el fan y el deporte, llevando el análisis estadístico a un nivel más profundo. "
    "Y a pesar de las lesiones, las derrotas y los momentos difíciles, la esencia del juego permanece: once contra once, luchando y soñando por avanzar una yarda más."

)


# Crear vocabulario de caracteres
chars = sorted(list(set(text))) # Todos los caractéres presentes en el texto ordenados
vocab_size = len(chars)


# Mapas para codificación
char2idx = {u: i for i, u in enumerate(chars)} # Asigna un número entre 0 y 43 (44 caracteres) a cada string en chars
idx2char = np.array(chars) #Da una arreglo de numpy con el string de cada caracter en las entradas
print(char2idx)
print(idx2char)


In [ ]:
# Funciones de codificación (usando el mapeo de codificación)
def encode(s): return np.array([char2idx[c] for c in s]) # Según el mapa de codificación char2idx codifica un string
print(encode('Cómo funciona encode'))
def decode(arr): return ''.join([idx2char[i] for i in arr])
print(decode([14, 55, 65, 0, 42, 57, 50, 39, 45, 51, 50, 37, 0, 40, 41, 39, 51, 40, 41]))

### Preparación de datos para el entrenamiento

Para entrenar al modelo, se divide el texto en **bloques de longitud fija**, conocida como **longitud de contexto** o `block_size`. Cada bloque consiste en una secuencia de tokens consecutivos del corpus.

El objetivo del modelo es **predecir el siguiente token** dado un contexto anterior. Por ello, el conjunto de entrenamiento se construye de la siguiente forma:

- Cada **input** es una secuencia de `block_size` tokens.
- Cada **target** (salida deseada) es la misma secuencia desplazada una posición hacia la izquierda, es decir, el token que sigue a cada uno de los tokens de entrada.

Por ejemplo, si se trabaja con bloques de longitud 50, el corpus se segmenta en pares input/target como sigue:

- **Input**: caracteres del índice `i` al `i+49` (50 caracteres).
- **Target**: caracteres del índice `i+1` al `i+50` (también 50 caracteres).

Esto permite que el modelo aprenda a predecir cada carácter basándose en el contexto anterior inmediato.

---

In [ ]:
# Secuencias
block_size = 50  # longitud del contexto
step = 1
inputs = []
targets = []

# Genera listas con 50 caracteres, cada lista es igual a la anterior con un caracter menos al principio y uno posterior al
for i in range(0, len(text) - block_size):
    inputs.append(encode(text[i:i+block_size]))
    targets.append(encode(text[i+1:i+block_size+1]))

print('Input 1:',decode(inputs[0]))
print("Target 1:",decode(targets[0]))
print('El input siguiente es igual al target previo')
print('Input 2:',decode(inputs[1]))
print('Target 2:',decode(targets[1]))

x = np.array(inputs)
y = np.array(targets)

## **Positional Encoding**

En los modelos Transformer, no se utilizan arquitecturas recurrentes ni convolucionales, por lo que es necesario proporcionar explícitamente información de **posición** al modelo para que pueda distinguir el orden de las entradas.

La clase `PositionalEncoding` implementa una codificación posicional senoidal descrita en el paper _"Attention is All You Need"_ (Vaswani et al., 2017). Esta codificación se suma a las entradas del decoder en este caso para incorporar información sobre el orden de las secuencias.

Para cada posición $ \text{pos} $ y cada dimensión $ i $ del embedding, se calcula:

$$
\text{PE}_{\text{pos}, 2i} = \sin\left(\frac{\text{pos}}{10000^{\frac{2i}{d_{\text{model}}}}}\right)
$$

$$
\text{PE}_{\text{pos}, 2i+1} = \cos\left(\frac{\text{pos}}{10000^{\frac{2i}{d_{\text{model}}}}}\right)
$$

Estas funciones sinusoidales de diferentes frecuencias permiten que el modelo:

- Distinga posiciones relativas y absolutas.
- Generalice a secuencias más largas que las vistas durante el entrenamiento.

# Ejemplo numérico: Entrada "Hola" al Transformer Decoder

## Tokens y IDs
| Token | ID  |
|-------|-----|
| 'H'   | 0   |
| 'o'   | 1   |
| 'l'   | 2   |
| 'a'   | 3   |

## Embeddings (dimensión = 2)
| ID  | Embedding          |
|-----|--------------------|
| 0   | [1.0, 0.5]         |
| 1   | [0.4, 1.2]         |
| 2   | [0.7, 0.3]         |
| 3   | [1.1, 0.9]         |

## Positional Encoding (fija con seno y coseno)

$$
\begin{cases}
PE_{(pos,0)} = \sin(pos) \\
PE_{(pos,1)} = \cos\left(\frac{pos}{100}\right)
\end{cases}
$$

| Posición | PE(pos,0) = sin(pos) | PE(pos,1) = cos(pos/100) |
|----------|---------------------|--------------------------|
| 0        | 0.0000              | 1.0000                   |
| 1        | 0.8415              | 0.9950                   |
| 2        | 0.9093              | 0.9801                   |
| 3        | 0.1411              | 0.9553                   |

## Suma Embedding + Positional Encoding

| Token | Embedding    | Positional Encoding | Suma (Input al decoder)     |
|-------|--------------|---------------------|-----------------------------|
| 'H'   | [1.0, 0.5]   | [0.0000, 1.0000]    | [1.0000 + 0.0000, 0.5 + 1.0] = [1.0000, 1.5000]   |
| 'o'   | [0.4, 1.2]   | [0.8415, 0.9950]    | [0.4 + 0.8415, 1.2 + 0.9950] = [1.2415, 2.1950]   |
| 'l'   | [0.7, 0.3]   | [0.9093, 0.9801]    | [0.7 + 0.9093, 0.3 + 0.9801] = [1.6093, 1.2801]   |
| 'a'   | [1.1, 0.9]   | [0.1411, 0.9553]    | [1.1 + 0.1411, 0.9 + 0.9553] = [1.2411, 1.8553]   |

---

**Resultado final (matriz de entrada al Transformer Decoder):**

$$
\begin{bmatrix}
1.0000 & 1.5000 \\
1.2415 & 2.1950 \\
1.6093 & 1.2801 \\
1.2411 & 1.8553
\end{bmatrix}
$$


---

In [ ]:
class PositionalEncoding(layers.Layer):
    def __init__(self, maxlen, d_model):
        super().__init__()
        pos = np.arange(maxlen)[:, np.newaxis]  # Posiciones (shape: [maxlen, 1])
        i = np.arange(d_model)[np.newaxis, :]   # Dimensiones del embedding (shape: [1, d_model])

        angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
        angle_rads = pos * angle_rates  # Producto de posición y frecuencia angular

        # Aplicar sin a índices pares (0, 2, 4, ...)
        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
        # Aplicar cos a índices impares (1, 3, 5, ...)
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

        self.pos_encoding = tf.cast(angle_rads[np.newaxis, ...], dtype=tf.float32)  # [1, maxlen, d_model]

    def call(self, x):
        # x: Tensor de entrada con shape [batch_size, seq_len, d_model]
        # Retorna la suma del embedding con la codificación posicional correspondiente
        return x + self.pos_encoding[:, :tf.shape(x)[1], :]

## **Causal Mask**

En el Transformer Decoder, se utiliza una **máscara causal (causal mask)** para evitar que un token "vea el futuro". Durante el entrenamiento de modelos de lenguaje es importante, ya que se busca que la predicción del siguiente token se base solo en los tokens anteriores o actuales, no en los futuros.

---

In [ ]:
def causal_mask(block_size):
    mask = tf.linalg.band_part(tf.ones((block_size, block_size)), -1, 0)  # 1s diagonal y abajo
    mask = tf.expand_dims(tf.expand_dims(mask, 0), 0)  # (1,1,block_size,block_size)
    return tf.cast(mask, tf.bool)

print(causal_mask(4))

### Verificación del funcionamiento de la máscara causal

Para comprobar cómo actúa la **máscara causal** al aplicarse en una capa de atención, se puede comparar la salida de la red con y sin dicha máscara.

Se procede de la siguiente manera:

1. **Se define una máscara sin restricciones**, es decir, una matriz donde todos los elementos son válidos (equivalente a permitir atención total entre todos los tokens).

2. **Se define una máscara causal**, donde solo se permite que cada token atienda a sí mismo y a los tokens anteriores en la secuencia (bloque inferior triangular).

3. Ambas máscaras se aplican a una misma entrada a través de una capa de atención.

4. Se comparan las **salidas (outputs)** o los **pesos de atención** generados por la capa.

---

#### Observación esperada

- Con la **máscara causal**, los pesos de atención asignados a posiciones futuras (a la derecha en la matriz) serán **cero**, lo que refleja que un token **no puede "ver hacia el futuro"**.
  
- Con la **máscara libre (sin restricciones)**, los tokens pueden atender libremente a cualquier posición, por lo tanto, **no hay ceros obligatorios** en la matriz de atención.

Esto permite verificar visualmente y numéricamente que la máscara causal está funcionando correctamente dentro del mecanismo de atención.

---

In [ ]:
# Máscara sin restricción (todo True)
def full_mask(block_size):
    mask = tf.ones((1, 1, block_size, block_size), dtype=tf.bool)
    return mask

block_size_test = 4
embed_dim_test = 4
num_heads_test = 2

mha = tf.keras.layers.MultiHeadAttention(num_heads=num_heads_test, key_dim=embed_dim_test // num_heads_test)

# Crear máscaras
mask = causal_mask(block_size_test)
mask_full = full_mask(block_size_test)

# Input fijo (batch=1, seq_len=4, embed_dim=4)
x_test = tf.constant([[[1., 1., 1., 1.],
                  [2., 2., 2., 2.],
                  [3., 3., 3., 3.],
                  [4., 4., 4., 4.]]])


# Atención con máscara causal y obtener scores
out_causal, attn_scores_causal = mha(x_test, x_test, attention_mask=mask, return_attention_scores=True)

# Atención con máscara full y obtener scores
out_full, attn_scores_full = mha(x_test, x_test, attention_mask=mask_full, return_attention_scores=True)

print("Pesos de atención con máscara causal (shape):", attn_scores_causal.shape)
print(attn_scores_causal.numpy())

print("\nPesos de atención sin máscara (shape):", attn_scores_full.shape)
print(attn_scores_full.numpy())

## **Definición del Modelo: `build_model`**

La función `build_model(seq_len, vocab_size)` construye un modelo autoregresivo basado en un **Transformer Decoder**. Su objetivo es predecir el siguiente token en una secuencia de texto.

El modelo recibe secuencias de longitud `seq_len`, donde cada token ha sido transformado en un índice entero perteneciente a un vocabulario de tamaño `vocab_size`.

### Componentes principales:

- **Embedding Layer**: Mapea cada índice del vocabulario a un vector de dimensión fija (`d_model`), lo que permite representar los tokens de manera continua.
  
- **Positional Encoding**: Como el Transformer no tiene una estructura secuencial inherente, se añade codificación posicional al embedding para introducir información del orden en la secuencia.

- **Causal Mask**:
  
  La máscara causal evita que el modelo "vea el futuro". Se implementa como una matriz triangular inferior que impide que cada token atienda a tokens posteriores a él.
  
  Esto es crucial en tareas de predicción secuencial, donde se quiere que el modelo aprenda a predecir el siguiente token sin tener acceso a los futuros.

- **MultiHeadAttention**: Permite al modelo enfocarse en diferentes partes de la secuencia en paralelo, respetando la máscara causal.Dado un conjunto de vectores de consulta (**Q**), clave (**K**) y valor (**V**), el mecanismo de atención escalada se calcula como:

$$
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{Q K^T}{\sqrt{d_k}} + \text{mask}\right) V
$$


- **Feed Forward Layer**: Dos capas densas que transforman las salidas del bloque de atención.

- **Dropout y Normalización**: Se incluyen capas de `Dropout` para evitar overfitting y `LayerNormalization` para estabilizar el entrenamiento.

- **Capa de salida**: Una capa `Dense` con `vocab_size` unidades y softmax, que predice la probabilidad de cada palabra del vocabulario como siguiente token.

---

In [ ]:
def build_model(vocab_size, block_size, d_model=64, num_heads=2, ff_dim=128):
    inputs = layers.Input(shape=(block_size,))
    x = layers.Embedding(input_dim=vocab_size, output_dim=d_model)(inputs)
    x = PositionalEncoding(block_size, d_model)(x)

    dropout_rate = 0.1  # El 10% de las neuronas se "desactivan" en las capas de dropout para evitar sobreajuste

    # Multi-head self attention con máscara causal
    mha = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
    mask_ = causal_mask(block_size)
    attn_out = mha(x, x, attention_mask=mask_)
    attn_out = layers.Dropout(dropout_rate)(attn_out)  # Dropout en salida de atención
    x = layers.Add()([x, attn_out]) #Suma elemento a elemento dos tensores: el tensor x original y el resultado de la atención attn_out.
    x = layers.LayerNormalization()(x)

    # Feed Forward
    ff_out = layers.Dense(ff_dim, activation='relu')(x)
    ff_out = layers.Dropout(dropout_rate)(ff_out)  # Dropout en salida de la primera densa
    ff_out = layers.Dense(d_model)(ff_out)
    ff_out = layers.Dropout(dropout_rate)(ff_out)  # Dropout en salida de la segunda densa
    x = layers.Add()([x, ff_out])
    x = layers.LayerNormalization()(x)

    outputs = layers.Dense(vocab_size, activation='softmax')(x)

    model = keras.Model(inputs, outputs)
    return model


## **Construcción y entrenamiento del modelo**

Este bloque de código crea, compila y entrena un modelo de red neuronal para tareas de predicción de secuencias (por ejemplo, generación de texto o modelado de lenguaje).

- `optimizer='adam'`: Se utiliza el optimizador Adam, una combinación de los métodos de descenso de gradiente con momento y RMSProp.

- `loss='sparse_categorical_crossentropy'`: Se emplea esta función de pérdida porque el modelo realiza una clasificación multiclase, donde el objetivo (target) es un índice entero que representa un token.

---

In [ ]:
model = build_model(vocab_size=vocab_size, block_size=block_size)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
model.summary()

In [ ]:
history = model.fit(x, y[..., np.newaxis], epochs=200, verbose=2) # El batch-size se establece como 32 si no se indica otra cosa
print("Entrenamiento finalizado.")

## **Función `generate_text`**

Esta función genera una secuencia de texto carácter por carácter (o token por token), utilizando el modelo entrenado. A partir de una cadena inicial (`start_string`), predice `num_chars` caracteres adicionales uno a uno.

### Parámetros de entrada:

- **`model`**: Modelo entrenado que recibe una secuencia de tokens de longitud fija (`block_size`) y predice los logits para el siguiente token.
- **`start_string`**: Cadena de texto inicial a partir de la cual se comienza a generar texto.
- **`num_chars`**: Número de caracteres (o tokens) a generar adicionalmente.

---


In [ ]:
def generate_text(model, start_string, num_chars=100):
    input_eval = encode(start_string)
    generated = list(input_eval)

    for _ in range(num_chars):
        # Preparar input (padded o recortado al tamaño del bloque)
        input_seq = generated[-block_size:]
        if len(input_seq) < block_size:
            input_seq = [0] * (block_size - len(input_seq)) + input_seq

        input_seq = np.array(input_seq)[np.newaxis, :]  # shape (1, block_size)
        preds = model.predict(input_seq, verbose=0)
        next_token_logits = preds[0, -1]

        # Elegir el siguiente token
        next_token = np.argmax(next_token_logits)


        generated.append(next_token)

    return decode(generated)

In [ ]:
seed1 = "Patrick Mahomes"
output1 = generate_text(model, seed1, num_chars=100)
seed2 = "americano"
output2 = generate_text(model, seed2, num_chars=100)
print('Prueba 2:',output1)
print('Prueba 1:',output2)

En Keras se puede guardar todo el modelo (estructura + pesos + optimizador)

Esto permite recargar el modelo completo después, tal como estaba:

In [ ]:
# Guardar el modelo completo
model.save_weights("NFL_GPT.keras")
# Cargar el modelo completo
modelo_cargado = load_model("NFL_GPT.keras")